In [159]:
import json
import re

import pandas as pd
pd.set_option('display.max_columns', None)
import requests
'''
LOOK THIS::: https://stackoverflow.com/questions/78307096/problem-how-to-scrape-dynamically-loaded-data-table-in-python
'''
url = "https://baseballsavant.mlb.com/leaderboard/statcast-park-factors"
url = "https://baseballsavant.mlb.com/leaderboard/statcast-park-factors?type=distance&year=2023&batSide=&stat=index_wOBA&condition=All&rolling="
url = "https://baseballsavant.mlb.com/leaderboard/statcast-park-factors?type=distance-all&year=2023&batSide=&stat=index_wOBA&condition=All&rolling="
response = requests.get(url)
data = re.search(r"data = (.*);", response.text).group(1)
data = json.loads(data)
df = pd.DataFrame(data)

engine = get_mlb_db_engine()

df.to_sql('ParkFactors', engine, if_exists='replace', index=False)

36

In [157]:
cur_year = '2023'
ser = pd.Series(df.columns)

# Iterate over the items in the Series
for idx, item in enumerate(ser):
    if cur_year in item:
        print(idx, item)
        return idx



5 extra_distance_2023


In [ ]:
def get_park_factor(backtest_year=''):
    


In [133]:
#df['avg_extra_distance'] = df[df.columns[4:]].mean(axis=1)
import numpy as np
filled_df = df[df.columns[4:]].fillna(np.nan)
filled_df = filled_df.astype(float)
df['avg_extra_distance'] = filled_df.mean(axis=1)
df.sort_values(by='avg_extra_distance', ascending=False)[['venue_name', 'avg_extra_distance']]                                         

,venue_name,avg_extra_distance
18,Coors Field,20.011111
22,Chase Field,8.977778
25,Globe Life Field,5.500000
24,Tropicana Field,4.588889
4,Choctaw Stadium,4.200000
21,Kauffman Stadium,3.111111
16,Truist Park,2.000000
35,American Family Field,1.744444
15,Great American Ball Park,1.088889
12,loanDepot park,0.666667


In [41]:
from src.data.data_utils import query_mlb_db, get_mlb_db_engine

In [19]:
date_range_df = query_mlb_db('select min(game_date) as min_date, max(game_date) as max_date from Statcast')

In [29]:
from datetime import datetime

min_dt = datetime.strptime(date_range_df['min_date'][0], '%Y-%m-%d %H:%M:%S.%f')
min_date = min_dt.strftime('%m/%d/%Y')

max_dt = datetime.strptime(date_range_df['max_date'][0], '%Y-%m-%d %H:%M:%S.%f')
max_date = max_dt.strftime('%m/%d/%Y')

print(min_date, max_date)

04/02/2017 05/17/2024


In [51]:
game_pks_df = query_mlb_db('select distinct game_pk from Statcast')
game_pks = game_pks_df['game_pk']

#', '.join(game_pks)
str_game_pks = ', '.join(str(i) for i in game_pks)

In [88]:
import json
import requests

res = requests.get('https://statsapi.mlb.com/api/v1/schedule?gamePk=632097')
game_json = res.json()

In [89]:
game_json.keys()

dict_keys(['messageNumber', 'message', 'timestamp', 'traceId'])

In [87]:
game_json['dates'][0]['games'][0]['venue']

{'id': 3309, 'name': 'Nationals Park', 'link': '/api/v1/venues/3309'}

In [98]:
from tqdm import tqdm
import time

sleep_time = 1/10
venue_data = []
for game_pk in tqdm(game_pks, desc="Fetching Game Data", total=len(game_pks)):
    res = requests.get(f'https://statsapi.mlb.com/api/v1/schedule?gamePk={game_pk}')
    if res.status_code == 200:
        game_json = res.json()
        venue = game_json['dates'][0]['games'][0]['venue']
        venue['game_pk'] = game_pk
        venue_data.append(venue)
        
        time.sleep(sleep_time)

Fetching Game Data: 100%|█████████████████████████████████████████████████████| 17360/17360 [1:22:42<00:00,  3.50it/s]


In [100]:
df = pd.DataFrame(venue_data)

In [102]:
df[['game_pk','name','id']]

,game_pk,name,id
0,490099,Busch Stadium,2889
1,490106,Tropicana Field,12
2,490110,Chase Field,15
3,490098,Citi Field,3289
4,490100,Globe Life Park in Arlington,13
...,...,...,...
17355,746641,Progressive Field,5
17356,746878,Wrigley Field,17
17357,747037,Oriole Park at Camden Yards,2
17358,747122,Truist Park,4705


In [103]:
engine = get_mlb_db_engine()

In [106]:
upload_df = df.rename(columns={"name": "venue_name", "id": "venue_id"})[['game_pk','venue_name','venue_id']]
upload_df

,game_pk,venue_name,venue_id
0,490099,Busch Stadium,2889
1,490106,Tropicana Field,12
2,490110,Chase Field,15
3,490098,Citi Field,3289
4,490100,Globe Life Park in Arlington,13
...,...,...,...
17355,746641,Progressive Field,5
17356,746878,Wrigley Field,17
17357,747037,Oriole Park at Camden Yards,2
17358,747122,Truist Park,4705


In [108]:
upload_df.to_sql('VenueGamePkMapping', engine, if_exists='replace', index=False)

17360

In [109]:
upload_df.to_csv('VenueMappingInit.csv')